In [1]:
# %%writefile core.py
import numba as nb
import pandas as pd
import numpy as np
from loguru import logger
import numexpr as ne
import sympy
import scipy
from math import * 
logger.info(f"Pandas version: {pd.__version__}")
logger.info(f"Numpy version: {np.__version__}")
logger.info(f"Numba version: {ne.__version__}")
logger.info(f"NumExpr version: {nb.__version__}")
logger.info(f"SymPy version: {sympy.__version__}")
logger.info(f"SciPy version: {scipy.__version__}")

def softplus_np(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    safe = x < 100
    return np.log(1 + np.exp(x * safe)) * safe + x * (1-safe)

def softplus_vec_wrap(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    @nb.vectorize('float32(float32)',target = "parallel")
    def softplus_nb(x):
        safe = x < 100
        return log(1 + exp(x * safe)) * safe + x * (1-safe)
    result = softplus_nb(x)
    return result

softplus_jit = nb.jit(softplus_np)
softplus_njit = nb.njit(softplus_np)
softplus_fastmath_njit = nb.njit(fastmath=True,parallel=True)(softplus_np)

2021-02-13 14:02:17.088 | INFO     | __main__:<module>:9 - Pandas version: 1.1.4
2021-02-13 14:02:17.093 | INFO     | __main__:<module>:10 - Numpy version: 1.18.5
2021-02-13 14:02:17.097 | INFO     | __main__:<module>:11 - Numba version: 2.7.1
2021-02-13 14:02:17.100 | INFO     | __main__:<module>:12 - NumExpr version: 0.50.1
2021-02-13 14:02:17.110 | INFO     | __main__:<module>:13 - SymPy version: 1.6.1
2021-02-13 14:02:17.115 | INFO     | __main__:<module>:14 - SciPy version: 1.5.0


In [3]:
params = {"n":2000000}
%time softplus_np(2000000)
%time softplus_jit(2000000)
%time softplus_njit(2000000)
%time softplus_fastmath_njit(2000000)
%time softplus_vec_wrap(**params)

Wall time: 221 ms
Wall time: 126 ms
Wall time: 136 ms
Wall time: 38.8 ms
Wall time: 274 ms


array([ 71.49738 , 110.246956,  74.97696 , ..., 135.8275  , 147.74281 ,
        74.883194], dtype=float32)

In [131]:
# @np.vectorize
# def softplus(x):
#     safe = x < 100
#     return log(1 + exp(x * safe)) * safe + x * (1-safe)

# @nb.vectorize('float32(float32)',target = "parallel")
# def softplus_nb(x):
#     safe = x < 100
#     return log(1 + exp(x * safe)) * safe + x * (1-safe)




array([113.215385,  73.7463  ,        inf,        inf, 118.672295,
       111.823944,  58.091507, 137.9336  ,  60.522278, 130.74448 ,
        71.602715, 107.65229 , 133.45897 ,        inf, 127.560646,
              inf,        inf, 115.60619 ,  70.40927 , 102.02923 ],
      dtype=float32)